# Conjunction AllenNLP BiLSTM

Load the needed packages: 

In [1]:
from allennlp_models.pretrained import load_predictor

/home/elena/anaconda3/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

Creating a prediction by the model

In [5]:
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("John met Max and smiled.")
output

{'verbs': [{'verb': 'met',
   'description': '[ARG0: John] [V: met] [ARG1: Max] and smiled .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O', 'O', 'O']},
  {'verb': 'smiled',
   'description': '[ARG0: John] met Max and [V: smiled] .',
   'tags': ['B-ARG0', 'O', 'O', 'O', 'B-V', 'O']}],
 'words': ['John', 'met', 'Max', 'and', 'smiled', '.']}

In [6]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [7]:
d = ["John met Max and smiled."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'met',
     'description': '[ARG0: John] [V: met] [ARG1: Max] and smiled .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O', 'O', 'O']},
    {'verb': 'smiled',
     'description': '[ARG0: John] met Max and [V: smiled] .',
     'tags': ['B-ARG0', 'O', 'O', 'O', 'B-V', 'O']}],
   'words': ['John', 'met', 'Max', 'and', 'smiled', '.']}],
 array([1.]))

In [8]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [9]:
def get_arg(pred, arg_target='ARG0'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [10]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [11]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [12]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg0

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

## Test for first predicate

In [13]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} met Max and smiled.", meta=True,nsamples=500, remove_duplicates=True)


In [14]:
with open('/datasets/conjunction_pred1_arg0_bilstm.txt', 'w') as f:
    print(t.data, file = f)

In [15]:
# initialize a test object
import csv
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred1_arg0_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 500 examples
Test cases:      500
Fails (rate):    0 (0.0%)
['Howard', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Simon', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Pamela', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Kathy', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Maria', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Nick', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Donna', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Marilyn', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Arthur', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Donna', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Diane', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Nick', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Sandra', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Edwin', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Ian', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Colin', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Elizabeth', 'met', 'Max', 'and', 'smiled', '.'] [ True]
['Bil

In [16]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("John met {first_name} and smiled.", meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/conjunction_pred1_arg1_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred1_arg1_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)


Predicting 500 examples
Test cases:      500
Fails (rate):    3 (0.6%)

Example fails:
[ARG0: John] [V: met] [ARGM-MNR: Patrick] and smiled .
----
[ARG0: John] [V: met] [ARGM-MNR: Patrick] and smiled .
----
[ARG0: John] [V: met] [ARGM-MNR: Patrick] and smiled .
----
['John', 'met', 'Alfred', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Jason', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Frances', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Katie', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Frank', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Harold', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Joan', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Don', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Roy', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Victoria', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Joseph', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Chris', 'and', 'smiled', '.'] [ True]
['John', 'met', 'Francis', 'and', 'smiled', '.'] [ True]
['John', 'met

In [17]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} met {first_name1} and smiled.", meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/conjunction_pred1_arg0_2firstname_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred1_arg0_2firstname_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 497 examples
Test cases:      497
Fails (rate):    1 (0.2%)

Example fails:
[ARGM-DIS: Andrea] [V: met] [ARG1: Dave] and smiled .
----
['Andrea', 'met', 'Dave', 'and', 'smiled', '.'] [False]
['Mary', 'met', 'Jessica', 'and', 'smiled', '.'] [ True]
['Jack', 'met', 'Sue', 'and', 'smiled', '.'] [ True]
['Alice', 'met', 'Dan', 'and', 'smiled', '.'] [ True]
['Karen', 'met', 'Philip', 'and', 'smiled', '.'] [ True]
['Ryan', 'met', 'Judy', 'and', 'smiled', '.'] [ True]
['Tom', 'met', 'Sarah', 'and', 'smiled', '.'] [ True]
['Charles', 'met', 'Ken', 'and', 'smiled', '.'] [ True]
['Anna', 'met', 'Kenneth', 'and', 'smiled', '.'] [ True]
['Arthur', 'met', 'Eric', 'and', 'smiled', '.'] [ True]
['Joan', 'met', 'Tim', 'and', 'smiled', '.'] [ True]
['Johnny', 'met', 'Christopher', 'and', 'smiled', '.'] [ True]
['Pamela', 'met', 'Jonathan', 'and', 'smiled', '.'] [ True]
['Emily', 'met', 'Andrew', 'and', 'smiled', '.'] [ True]
['Don', 'met', 'Margaret', 'and', 'smiled', '.'] [ True]
['Susan', 

In [18]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name1']])
    arg_0 = get_arg(pred, arg_target='ARG1')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [19]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} met {first_name1} and smiled.", meta=True,nsamples=500, remove_duplicates=True)
with open('/datasets/conjunction_pred1_arg1_2firstname2_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred1_arg1_2firstname2_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 496 examples
Test cases:      496
Fails (rate):    6 (1.2%)

Example fails:
[ARG0: Alexandra] [V: met] [ARGM-MNR: Kathy] and smiled .
----
[ARG0: Alice] [V: met] [ARGM-MNR: Patrick] and smiled .
----
[ARG0: Jason] [V: met] [ARGM-MNR: Kathy] and smiled .
----
['Sara', 'met', 'Katie', 'and', 'smiled', '.'] [ True]
['Jimmy', 'met', 'Al', 'and', 'smiled', '.'] [ True]
['Donna', 'met', 'James', 'and', 'smiled', '.'] [ True]
['Evelyn', 'met', 'Heather', 'and', 'smiled', '.'] [ True]
['Paul', 'met', 'Julie', 'and', 'smiled', '.'] [ True]
['Stephanie', 'met', 'Anna', 'and', 'smiled', '.'] [ True]
['Claire', 'met', 'Andrew', 'and', 'smiled', '.'] [ True]
['Benjamin', 'met', 'Lynn', 'and', 'smiled', '.'] [ True]
['Andrew', 'met', 'Angela', 'and', 'smiled', '.'] [ True]
['Samuel', 'met', 'Arthur', 'and', 'smiled', '.'] [ True]
['Lisa', 'met', 'Sally', 'and', 'smiled', '.'] [ True]
['Diane', 'met', 'Rachel', 'and', 'smiled', '.'] [ True]
['James', 'met', 'Thomas', 'and', 'smiled', '.'] 

## Test for second predicate

In [20]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description']

In [21]:
def get_arg(pred, arg_target='ARG0'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [22]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']
        
    return predicate_structure

In [23]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} met Max Mustermann and smiled.", meta=True,nsamples=500, remove_duplicates=True)
# initialize a test object
with open('/datasets/conjunction_pred2_arg0_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/conjunction_pred2_arg0_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 500 examples
Test cases:      500
Fails (rate):    1 (0.2%)

Example fails:
Diane met Max Mustermann and [V: smiled] .
----
['Carl', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Frances', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Ron', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Carl', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Catherine', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Caroline', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Eleanor', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Matt', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Linda', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Larry', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Kathleen', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Robert', 'met', 'Max', 'Mustermann', 'and', 'smiled', '.'] [ True]
['Heather', 'met', 'Max', 'Mustermann',

# End of Notebook